In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer  # для того щоб представити текстові дані у вигляді вектора
from sklearn.metrics.pairwise import cosine_similarity  # для cosine similarity
import difflib  # для пошуку максимально схожих слів

## Cosine Similarity
**Косинус подібності** - коефіцієнт подібності двох не нульових векторів у предгільбертовому просторі, який обчислюється як косинус кута між ними. Чим менше значення кута, тим більше схожими є два вектори.
_Формула для обчислення косинуса подібності_ <br>
$cos(\alpha) = \frac{A \times B}{\|A\|\|B\|}$

#### Завантажуємо дані

In [2]:
movie = pd.read_csv("movie.csv")
movie["index"] = [i for i in range(len(movie))]
movie

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj,index
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,1.379999e+08,1.392446e+09,0
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,1.379999e+08,3.481613e+08,1
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,1.012000e+08,2.716190e+08,2
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015,1.839999e+08,1.902723e+09,3
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015,1.747999e+08,1.385749e+09,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10861,21,tt0060371,0.080598,0,0,The Endless Summer,Michael Hynson|Robert August|Lord 'Tally Ho' B...,NaN,Bruce Brown,NaN,...,95,Documentary,Bruce Brown Films,6/15/1966,11,7.4,1966,0.000000e+00,0.000000e+00,10861
10862,20379,tt0060472,0.065543,0,0,Grand Prix,James Garner|Eva Marie Saint|Yves Montand|Tosh...,NaN,John Frankenheimer,Cinerama sweeps YOU into a drama of speed and ...,...,176,Action|Adventure|Drama,Cherokee Productions|Joel Productions|Douglas ...,12/21/1966,20,5.7,1966,0.000000e+00,0.000000e+00,10862
10863,39768,tt0060161,0.065141,0,0,Beregis Avtomobilya,Innokentiy Smoktunovskiy|Oleg Efremov|Georgi Z...,NaN,Eldar Ryazanov,NaN,...,94,Mystery|Comedy,Mosfilm,1/1/1966,11,6.5,1966,0.000000e+00,0.000000e+00,10863
10864,21449,tt0061177,0.064317,0,0,"What's Up, Tiger Lily?",Tatsuya Mihashi|Akiko Wakabayashi|Mie Hama|Joh...,NaN,Woody Allen,WOODY ALLEN STRIKES BACK!,...,80,Action|Comedy,Benedict Pictures Corp.,11/2/1966,22,5.4,1966,0.000000e+00,0.000000e+00,10864


#### Виділяємо необхідні фактори для порівняння

In [3]:
features = ['keywords','cast','genres','director','tagline']

#### Заповнюємо всі пропущені дані порожніми рядками

In [4]:
for feature in features:
    movie[feature] = movie[feature].fillna('')

In [5]:
def combine_features(row):
    '''
        Допоміжна функція для того, щоб
        створити єдиний рядок який містить
        слова з усіх факторів
    '''
    try:
        return row['keywords'] +" "+row['cast']+" "+row['genres']+" "+row['director']+" "+row['tagline']
    except:
        print ("Error:", row)


за допомогою методу **apply** застосовуємо функцію
**combine_features** для кожного рядку і формуємо новий стовпчик в датасеті

In [6]:
movie["combined_features"] = movie.apply(combine_features,axis=1)

In [7]:
def title_from_index(index):
    '''
        Допоміжна функція яка дістає
        тайтл фільму за стовпчиком index
    '''
    return movie[movie.index == index]["original_title"].values[0]


In [8]:
def index_from_title(original_title):
    '''
        Допоміжна функція яка дістає
        index фільму за стовпчиком original_title
    '''
    title_list = movie['original_title'].tolist()
    common = difflib.get_close_matches(original_title, title_list, 1)  # шукаємо найближчу схожу назву
    titlesim = common[0]
    return movie[movie.original_title == titlesim]["index"].values[0]


### Загальний опис того що відбувається далі
1. **fit_transform** - ця функція трансформує кожен рядок в вектор, при цьому враховуючи всі інші слова в стопці датасеті, при цьому виходить дуже довгий вектор з 0 та 1.
2. **cosine_sim** - ця функція отримує на вхід **матрицю з векторів**(яка містить нулі та одиниці) і порівнює вектори попарно, звідси маємо на головній діагонлі отриманої матриці одиниці(фільм порівнювали сам з собою), і інші числа, тобто отримуємо **оцінку від нуля до одиниці** наскільки фільми схожі за **кастом**, **хто їх знімав**, **жанри**, **ключові слова**, **теги** (насправді сюди можна прикрутити і друзів, типу якщо фільм дивився друг то вписувати його ім'я в стовпчик, якщо ні, то залишати поле пустим).

In [9]:
cv = CountVectorizer()  # Furious 7
count_matrix = cv.fit_transform(movie["combined_features"])  # це щось типу onehot
cosine_sim = cosine_similarity(count_matrix)
cosine_sim

array([[1.        , 0.18184824, 0.14037248, ..., 0.        , 0.03940552,
        0.        ],
       [0.18184824, 1.        , 0.21442251, ..., 0.        , 0.04012862,
        0.03513642],
       [0.14037248, 0.21442251, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.04662524,
        0.        ],
       [0.03940552, 0.04012862, 0.        , ..., 0.04662524, 1.        ,
        0.03806935],
       [0.        , 0.03513642, 0.        , ..., 0.        , 0.03806935,
        1.        ]])

In [56]:
user_movie = input("Enter movie of your choice:\t")
movie_index = index_from_title(user_movie)


Enter movie of your choice:	Furious 7


In [62]:
similar_movies =  list(enumerate(cosine_sim[movie_index]))  # дістаємо вектор з значеннями від [0, 1] номеруємо їх як в index
similar_movies_sorted = sorted(similar_movies,  key=lambda x:x[1],  reverse=True)  # сортуємо за значенням косинуса
print("\nOther movies you might be interested in:\n")

i=0
for rec_movie in similar_movies_sorted:
        if(i != 0):
            print(i, ") ", title_from_index(rec_movie[0]) sep="")
        i=i+1
        if i > 50:
            break



Other movies you might be interested in:

1) Fast & Furious 6
2) The Fast and the Furious
3) 2 Fast 2 Furious
4) The Fast and the Furious: Tokyo Drift
5) Death Race
6) Fast & Furious
7) Fast Five
8) Los Bandoleros
9) The Bank Job
10) Gone In Sixty Seconds
11) Turbo Charged Prelude to 2 Fast 2 Furious
12) War
13) The Final Destination
14) Saw
15) The Transporter Refueled
16) The Transporter
17) Need for Speed
18) Transit
19) Joshua Tree
20) Thief
21) Tau man ji D
22) Gone in 60 Seconds
23) Vehicle 19
24) Riddick
25) Highwaymen
26) London to Brighton
27) Safe
28) Death Sentence
29) White Sands
30) Bullet to the Head
31) Rolling Thunder
32) A Faster Horse
33) Back to the Future Part II
34) Pain & Gain
35) Grand Prix
36) The Outsider
37) Crank: High Voltage
38) ì§íŒ¨
39) The One
40) Curve
41) Turbo
42) Find Me Guilty
43) Out for a Kill
44) S.W.A.T.
45) Faster
46) Setup
47) xXx
48) The Hard Corps
49) Homefront
50) Machete Kills
